对大模型调用错误的记录进行查询，尝试进行重试

In [3]:
import pandas as pd


def load_excel(file_path: str) -> pd.DataFrame:
    """
    读取Excel文件，验证必要列并处理空值
    要求文件必须包含"商品ID"和"商品名称"列（可修改required_cols适配实际列名）
    """
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        required_cols = ['商品ID', '商品名称']
        # 检查必要列是否存在
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"缺少必要列：{', '.join(missing_cols)}，需包含{required_cols}")
        # 去除商品ID或名称为空的无效行
        df = df.dropna(subset=required_cols).reset_index(drop=True)
        # 强制转为字符串类型，避免数字ID/名称拼接出错（如科学计数法、格式丢失）
        df['商品ID'] = df['商品ID'].astype(str).str.strip()
        df['商品名称'] = df['商品名称'].astype(str).str.strip()
        return df
    except Exception as e:
        print(f"读取文件{file_path}失败：{str(e)}")
        raise

In [4]:
import math
import pandas as pd
import re


def pandas_str_to_series(s) -> pd.Series:
    # s为None或""或nan
    if s is None or s == "" or (isinstance(s, float) and math.isnan(s)):
        return None

    inner = s[s.find("(") + 1: s.rfind(")")]

    pattern = re.compile(r"(\w+)=('[^']*'|[^,]*)")
    data = {k: v.strip("'") if v.strip() != "nan" else None for k, v in pattern.findall(inner)}

    # 3️⃣ 转为 DataFrame
    df = pd.DataFrame([data])
    return df.iloc[0]



In [5]:

# 加载数据表文件
df = load_excel("../../output/补充Top3相似商品7b5f9635-c6a9-427b-b13e-e0bfb702273a.xlsx")
df.head()

,商品ID,是否已打标,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,...,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid,相似商品1（ID-名称）,相似商品2（ID-名称）,相似商品3（ID-名称）,相似商品
0,22639131950,0,七度空间 420mm绢爽网面透气超特长夜用卫生巾 4片_包,4片*1包,6.903244e+12,12.50,12.50,NaN,0,女性护理,...,NaN,NaN,NaN,NaN,卫生巾/卫生棉,41432125836,"Pandas(商品ID='10908978937', 商品名称='【苏菲】超熟睡柔棉感纤巧夜...","Pandas(商品ID='21326884399', 商品名称='苏菲420mm超熟睡柔棉感...","Pandas(商品ID='10907930090', 商品名称='【苏菲】超熟睡超薄裸感肌夜...",error
1,22652416416,1,胖东来 12°P精酿小麦啤酒 330ml_听,330ml*1听,6.972529e+12,8.32,8.49,NaN,1,酒类专区,...,NaN,NaN,NaN,NaN,精酿啤酒,41454728881,"Pandas(商品ID='17448439267', 商品名称='【爆品精酿】胖东来DL 1...",NaN,NaN,"Pandas(商品ID='17448439267', 商品名称='【爆品精酿】胖东来DL 1..."
2,22639565169,0,清风 纯水湿巾 10片_包,10片*1包,6.922266e+12,5.28,5.39,NaN,0,生活用纸,...,NaN,NaN,NaN,NaN,湿纸巾/湿厕纸,41431605679,"Pandas(商品ID='10908591313', 商品名称='清风 柔湿巾 清风湿巾纸纯...",NaN,NaN,"Pandas(商品ID='10908591313', 商品名称='清风 柔湿巾 清风湿巾纸纯..."
3,22637741974,0,彩色长尾夹燕尾夹19mm(40只)_盒,NaN,6.941245e+12,8.70,8.70,NaN,0,办公学习,...,NaN,NaN,NaN,NaN,夹子,41428383687,"Pandas(商品ID='19029840765', 商品名称='得力 8552ES 41m...","Pandas(商品ID='10908552330', 商品名称='得力8487长尾夹5号19...","Pandas(商品ID='10909128477', 商品名称='得力 彩色工字钉 35只_...","Pandas(商品ID='10908552330', 商品名称='得力8487长尾夹5号19..."
4,22637319896,0,家有喜事喜庆红色塑料盆 结婚嫁妆大红脸盆洗脚盆红妆喜盆_个 规格可选,35*11.5cm#37*12cm#33*11cm,6.938576e+12,5.24,5.24,NaN,0,家庭清洁,...,NaN,NaN,NaN,NaN,脸盆/水桶,41430419297,"Pandas(商品ID='10906639901', 商品名称='颜色随机 洗脸盆家用塑料盆...","Pandas(商品ID='10908552583', 商品名称='暗酒红色 格纹花纹男士结婚...","Pandas(商品ID='22419552653', 商品名称='礼花筒结婚礼炮筒开业乔迁彩...",NaN


In [6]:
# 提取相似商品列值为error的记录，相似商品列可能为None
error_rows = df[df['相似商品'].notnull() & df['相似商品'].str.contains('error')]
error_rows

,商品ID,是否已打标,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,...,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid,相似商品1（ID-名称）,相似商品2（ID-名称）,相似商品3（ID-名称）,相似商品
0,22639131950,0,七度空间 420mm绢爽网面透气超特长夜用卫生巾 4片_包,4片*1包,6.903244e+12,12.50,12.50,NaN,0,女性护理,...,NaN,NaN,NaN,NaN,卫生巾/卫生棉,41432125836,"Pandas(商品ID='10908978937', 商品名称='【苏菲】超熟睡柔棉感纤巧夜...","Pandas(商品ID='21326884399', 商品名称='苏菲420mm超熟睡柔棉感...","Pandas(商品ID='10907930090', 商品名称='【苏菲】超熟睡超薄裸感肌夜...",error
17,22637998436,1,元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶,500ml*1瓶,6.975177e+12,6.85,6.99,NaN,0,饮料专区,...,NaN,NaN,NaN,NaN,植物饮料,41429718457,"Pandas(商品ID='18828350974', 商品名称='元气森林 自在水 0糖0脂...",NaN,NaN,error
28,22640442876,0,斧头牌 晶怡厨房重油污净 500g_瓶,500g*1瓶,6.920722e+12,19.35,19.35,NaN,0,家庭清洁,...,NaN,NaN,NaN,NaN,多功能/万用清洁剂,41431887838,"Pandas(商品ID='19842173159', 商品名称='威猛先生 厨房重油污净 5...","Pandas(商品ID='15196627393', 商品名称='老管家厨房重油污清洗剂去污...","Pandas(商品ID='10907345818', 商品名称='威猛先生 管道疏通剂强力溶...",error
31,22636514970,0,统一UNI-PRESIDENT 低糖含有茶多酚绿茶茉莉味茶饮料 500毫升_瓶,500毫升*1瓶,6.925304e+12,3.42,3.49,NaN,2,饮料专区,...,NaN,NaN,NaN,NaN,茶饮料,41430928029,"Pandas(商品ID='10908552830', 商品名称='统一 茉莉味绿茶 500m...",NaN,NaN,error
33,22638074364,0,重型美工刀规格可选 工具架黑刃裁纸刀工业用多功能手工小刀片墙纸刀壁纸刀~,美工刀一把【内含合金刀片5枚】#合金刀片10枚,NaN,10.79,10.79,NaN,0,办公学习,...,NaN,NaN,NaN,NaN,美工刀,41431412131,"Pandas(商品ID='15019005365', 商品名称='得力 2034美工刀 不锈...","Pandas(商品ID='10908978865', 商品名称='得力 金属美工刀小号裁纸壁...","Pandas(商品ID='10908743392', 商品名称='得力 美工刀2041壁纸刀...",error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,22639760796,0,【密封锁鲜】 白色家用饺子盒食品冰箱专用冷冻盒密封保鲜多层速冻馄饨水饺收纳盒 1_个,白色一层,NaN,9.29,9.29,NaN,1,厨具餐具,...,NaN,NaN,NaN,NaN,保鲜盒,41429676874,"Pandas(商品ID='15017781650', 商品名称='【保鲜收纳盒】食品冰箱肉类...","Pandas(商品ID='17148417223', 商品名称='保鲜盒食品收纳盒塑料盒子密...","Pandas(商品ID='10908552070', 商品名称='玻璃饭盒 微波炉加热专用上...",error
475,22637822453,0,喇叭口小腿袜日系针织腿套52cm_双 jk堆堆袜软妹袜套千禧原宿显瘦袜套,白色-喇叭袜套,NaN,23.37,23.37,NaN,0,内裤衣袜,...,NaN,NaN,NaN,NaN,袜子,41430619415,"Pandas(商品ID='15018035961', 商品名称='针织jk套袜 小腿堆堆袜 ...","Pandas(商品ID='15018039861', 商品名称='超薄小腿袜 显瘦黑丝袜女夏...","Pandas(商品ID='18850537564', 商品名称='JK小腿袜中高筒ins潮纯...",error
478,22638628659,0,豹纹金钻辣妹穿戴甲30片_盒 高级感辣妹豹纹水钻法式手工穿戴甲,斑纹金钻穿戴甲#30片/盒,NaN,15.00,15.00,NaN,0,美发美甲,...,NaN,NaN,NaN,NaN,美甲工具,41429595748,"Pandas(商品ID='10907426856', 商品名称='网红新款ins Hs508...","Pandas(商品ID='10975775921', 商品名称='十月结晶 母乳保鲜袋 30...","Pandas(商品ID='15018116533', 商品名称='婵朵美甲 穿戴甲手工专用甲...",error
479,22638034168,0,假下眼睫毛纹身贴 女仿真下睫毛持久放水化妆一次性疤痕玩具防水睫毛纹身贴,YJM-01-10*14CM,NaN,3.82,3.82,NaN,0,美妆护肤,...,NaN,NaN,NaN,NaN,假睫毛及工具,41431004175,"Pandas(商品ID='10907426854', 商品名称='海氏海诺 1010cm敷贴...","Pandas(商品ID='12453567787', 商品名称='兔奶奶 假睫毛胶水无痕定型...",NaN,error


In [7]:
# error_rows数据信息，多少行，多少列
error_rows.shape

(84, 30)

In [9]:
from Limter import RateLimiter

# 设置限流器
limiter = RateLimiter()

In [14]:
# 处理单条 error 记录的协程
from llm_match import process_llm_row, llm_rank


async def handle_error_row(index, row, limiter, df):

    await limiter.record_call(800)

    similar_product1 = pandas_str_to_series(row['相似商品1（ID-名称）'])
    similar_product2 = pandas_str_to_series(row['相似商品2（ID-名称）'])
    similar_product3 = pandas_str_to_series(row['相似商品3（ID-名称）'])

    top1 = similar_product1['商品名称'] if similar_product1 is not None else ""
    top2 = similar_product2['商品名称'] if similar_product2 is not None else ""
    top3 = similar_product3['商品名称'] if similar_product3 is not None else ""

    # 模型排序
    idx = await llm_rank(row.商品名称, top1, top2, top3)
    print(f"  匹配结果：{idx}")

    if idx == 1:
        df.at[index, '相似商品'] = row['相似商品1（ID-名称）']
    elif idx == 2:
        df.at[index, '相似商品'] = row['相似商品2（ID-名称）']
    elif idx == 3:
        df.at[index, '相似商品'] = row['相似商品3（ID-名称）']
    elif idx == 0:
        df.at[index, '相似商品'] = None
    else:
        df.at[index, '相似商品'] = 'error'  # 仍然是错误，等待下一轮 retry



In [15]:
import asyncio


# 每一轮重试协程批处理
async def run_retry_batch(df, limiter, batch_size=10):
    tasks = []
    for index, row in df.iterrows():
        # 判断是否是error记录，如果不是跳过这条记录
        if row['相似商品'] != 'error':
            continue
        tasks.append(handle_error_row(index, row, limiter, df))
        if len(tasks) >= batch_size:
            await asyncio.gather(*tasks)
            tasks = []
    if tasks:
        await asyncio.gather(*tasks)

In [16]:
# 循环过滤 + 重试
async def retry_error_rows(df, limiter, max_rounds=5, batch_size=10):
    """
    df: 原始 DataFrame，包含 '相似商品' 字段
    max_rounds: 最多重试几轮
    batch_size: 每次协程批处理大小
    """

    for round_num in range(1, max_rounds + 1):
        # 过滤需要 retry 的记录
        error_rows = df[df['相似商品'] == 'error']

        retry_count = len(error_rows)
        print(f"\n====== 第 {round_num} 次遍历，需重试记录：{retry_count} 条 ======")

        if retry_count == 0:
            print("所有错误记录已处理完成！")
            break

        await run_retry_batch(df, limiter, batch_size=batch_size)

    else:
        print("\n⚠ 达到最大重试次数，但仍有未修复的错误记录")

In [17]:
await retry_error_rows(df, limiter, 10, 10)


====== 第 1 次遍历，需重试记录：84 条 ======
  正在处理错误记录：0 - 七度空间 420mm绢爽网面透气超特长夜用卫生巾 4片_包
  正在处理错误记录：17 - 元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶
  正在处理错误记录：28 - 斧头牌 晶怡厨房重油污净 500g_瓶
  正在处理错误记录：31 - 统一UNI-PRESIDENT 低糖含有茶多酚绿茶茉莉味茶饮料 500毫升_瓶
  正在处理错误记录：33 - 重型美工刀规格可选 工具架黑刃裁纸刀工业用多功能手工小刀片墙纸刀壁纸刀~
  正在处理错误记录：60 - Bakerdream百钻 面包糠 100g_袋
  正在处理错误记录：67 - 卫龙亲嘴烧 经典香辣风味调味面制品 24g_袋
  正在处理错误记录：68 - 无漆无蜡筷子10双装_盒 无节本色竹筷吸塑盒装碳化筷子
  正在处理错误记录：71 - 元气森林 0糖0卡0咖啡因菊花茶植物饮料 600ml_瓶
  正在处理错误记录：83 - 得力 24色可洗水彩笔 24支_盒
正在处理： 七度空间 420mm绢爽网面透气超特长夜用卫生巾 4片_包
正在处理： 元气自在水 0糖0脂红豆薏米水植物饮料 500ml_瓶
正在处理： 斧头牌 晶怡厨房重油污净 500g_瓶
正在处理： 统一UNI-PRESIDENT 低糖含有茶多酚绿茶茉莉味茶饮料 500毫升_瓶
正在处理： 重型美工刀规格可选 工具架黑刃裁纸刀工业用多功能手工小刀片墙纸刀壁纸刀~
正在处理： Bakerdream百钻 面包糠 100g_袋
正在处理： 卫龙亲嘴烧 经典香辣风味调味面制品 24g_袋
正在处理： 无漆无蜡筷子10双装_盒 无节本色竹筷吸塑盒装碳化筷子
正在处理： 元气森林 0糖0卡0咖啡因菊花茶植物饮料 600ml_瓶
正在处理： 得力 24色可洗水彩笔 24支_盒
  匹配结果：1
  匹配结果：0
  匹配结果：0
  匹配结果：0
  匹配结果：1
  匹配结果：0
  匹配结果：0
  匹配结果：2
商品：重型美工刀规格可选 工具架黑刃裁纸刀工业用多功能手工小刀片墙纸刀壁纸刀~ 发生异常: 1 validation error for RankSelect
  Invalid JSON: expected value at line 

In [18]:
# 提取相似商品列值为error的记录，相似商品列可能为None
error_rows = df[df['相似商品'].notnull() & df['相似商品'].str.contains('error')]
error_rows

,商品ID,是否已打标,商品名称,规格,条码,折扣价,原价,活动,销售,店内一级分类,...,三级分类json,tag,美团一级分类,美团二级分类,美团三级分类,skuid,相似商品1（ID-名称）,相似商品2（ID-名称）,相似商品3（ID-名称）,相似商品


In [20]:
# 导出excel
df.to_excel("../../output/补充Top3相似商品.xlsx", index=False)